# Load modules

In [43]:
import os
import folium

# Choose $t_0$

In [44]:
year = '2024'
month = '09'
day = '20'
hour = '03'
minute = '00'

In [45]:
# === Define bounding box and map center ===
TARGET_DOMAIN_LAT_MIN, TARGET_DOMAIN_LAT_MAX = -49, 31
TARGET_DOMAIN_LON_MIN, TARGET_DOMAIN_LON_MAX = -24, 60

bounds = [
    [TARGET_DOMAIN_LAT_MIN, TARGET_DOMAIN_LON_MIN],  # South-West
    [TARGET_DOMAIN_LAT_MAX, TARGET_DOMAIN_LON_MAX]   # North-East
]

map_center = [
    (TARGET_DOMAIN_LAT_MIN + TARGET_DOMAIN_LAT_MAX) / 2,
    (TARGET_DOMAIN_LON_MIN + TARGET_DOMAIN_LON_MAX) / 2
]

# === Create base map ===
m = folium.Map(
    location=map_center,
    zoom_start=4,
    tiles=None,
    max_bounds=True,
    min_zoom=3,
    max_zoom=8
)

m.fit_bounds(bounds)

# === Add only the required base layers ===

# 1. Light (default)
folium.TileLayer(
    tiles='CartoDB positron',
    name='Light',
    show=True,
    attr='©OpenStreetMap contributors, ©CartoDB'
).add_to(m)

# 2. Dark
folium.TileLayer(
    tiles='CartoDB dark_matter',
    name='Dark',
    show=False,
    attr='©OpenStreetMap contributors, ©CartoDB'
).add_to(m)

# 3. Esri Topo
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
    name='Esri Topo',
    show=False,
    attr='Tiles © Esri'
).add_to(m)

# 4. Esri Satellite
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    name="Esri Satellite",
    show=False,
    attr="Tiles © Esri — Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, etc.",
    overlay=False,
    control=True
).add_to(m)

# 5. OpenStreetMap
folium.TileLayer(
    tiles='OpenStreetMap',
    name='OpenStreetMap',
    show=False
).add_to(m)

# === Define image paths ===
outdir = f"tmp_vis/{year}{month}{day}_{hour}{minute}"
img_pred_t1 = os.path.join(outdir, "pred_t1.png")
img_pred_t3 = os.path.join(outdir, "pred_t3.png")
img_gt_t1   = os.path.join(outdir, "gt_t1.png")
img_gt_t3   = os.path.join(outdir, "gt_t3.png")

# === Add overlays ===
folium.raster_layers.ImageOverlay(
    name="Nowcast t+1h",
    image=img_pred_t1,
    bounds=bounds,
    opacity=0.6,
    show=False
).add_to(m)

folium.raster_layers.ImageOverlay(
    name="Nowcast t+3h",
    image=img_pred_t3,
    bounds=bounds,
    opacity=0.6,
    show=False
).add_to(m)

folium.raster_layers.ImageOverlay(
    name="Ground Truth t+1h",
    image=img_gt_t1,
    bounds=bounds,
    opacity=0.6,
    show=True
).add_to(m)

folium.raster_layers.ImageOverlay(
    name="Ground Truth t+3h",
    image=img_gt_t3,
    bounds=bounds,
    opacity=0.6,
    show=False
).add_to(m)

# === Layer control (expanded) ===
folium.LayerControl(collapsed=False).add_to(m)

# === Add colorbar legend ===
legend_html = f"""
<div style="
    position: fixed;
    bottom: 30px;
    left: 30px;
    width: 340px;
    height: 100px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
<b>Core Probability</b><br>
<img src="{os.path.basename(os.path.join(outdir, 'colorbar.png'))}" width="320px" height="10px"><br>
<div style="display: flex; justify-content: space-between; width: 320px;">
  <span>0.01</span>
  <span>0.2</span>
  <span>0.4</span>
  <span>0.6</span>
  <span>0.8</span>
  <span>1.0</span>
</div>
<i>Only values ≥ 0.01 are shown on map</i>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# === Add date/time annotation near the legend ===
datetime_html = f"""
<div style="
    position: fixed;
    bottom: 150px;
    left: 30px;
    width: 340px;
    height: 40px;
    background-color: rgba(255, 255, 255, 0.85);
    border: 2px solid grey;
    z-index: 9999;
    font-size: 15px;
    font-weight: bold;
    padding: 8px;
">
🕒 Nowcast origin t: {year}-{month}-{day} {hour}:{minute} UTC
</div>
"""
m.get_root().html.add_child(folium.Element(datetime_html))

# === Save map to file ===
output_path = os.path.join(outdir, f"nowcast_map_{year}{month}{day}_{hour}{minute}.html")
m.save(output_path)

print(f"✅ Map saved: {output_path}")

✅ Map saved: tmp_vis/20240920_0300/nowcast_map_20240920_0300.html
